# Dengue Cases Prediction using XGBoost and SHAP

This notebook demonstrates the process of predicting dengue cases using the `dengue_data_with_weather.csv` dataset. The analysis includes data preprocessing, model training with XGBoost, evaluation, and model interpretation using SHAP values.

**Note:** Please refer to the `ML_Assignment_Guidelines.pdf` for specific instructions on evaluation metrics and split criteria if they differ from standard practices.

In [ ]:
# Install necessary libraries if not already installed
!pip install xgboost shap pandas numpy matplotlib seaborn scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import shap
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Set plot style
sns.set(style="whitegrid")

## 1. Load Data

In [ ]:
# Load the dataset
df = pd.read_csv('dengue_data_with_weather_data.csv')

# Display the first few rows
display(df.head())

# Display dataset info
print(df.info())

# Check for missing values
print("\nMissing values:\n", df.isnull().sum())

## 2. Preprocessing

Handling missing values and encoding categorical variables.

In [ ]:
# Drop rows where target variable 'Cases' is missing
df = df.dropna(subset=['Cases'])

# Handle missing weather data
# Strategy: Fill with median of the column or interpolate if time-series continuity is assumed per district
# Here we use interpolation for a smoother fill if data is ordered by time, otherwise median
df['Temp_avg'] = df['Temp_avg'].interpolate(method='linear')
df['Precipitation_avg'] = df['Precipitation_avg'].interpolate(method='linear')
df['Humidity_avg'] = df['Humidity_avg'].interpolate(method='linear')

# Alternatively, fill remaining NaNs with column median
df = df.fillna(df.median(numeric_only=True))

# Check if any missing values remain
print("Missing values after handling:\n", df.isnull().sum())

In [ ]:
# Encoding Categorical Variables
# 'Province' and 'District' are categorical. We use Label Encoding for tree-based models like XGBoost.

le_province = LabelEncoder()
df['Province_Encoded'] = le_province.fit_transform(df['Province'])

le_district = LabelEncoder()
df['District_Encoded'] = le_district.fit_transform(df['District'])

# Ensure 'Year' and 'Month' are integers
df['Year'] = df['Year'].astype(int)
df['Month'] = df['Month'].astype(int)

print("Data Types:\n", df.dtypes)

## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Distribution of Cases
plt.figure(figsize=(10, 6))
sns.histplot(df['Cases'], bins=30, kde=True)
plt.title('Distribution of Dengue Cases')
plt.xlabel('Cases')
plt.ylabel('Frequency')
plt.show()

# Correlation Matrix
plt.figure(figsize=(12, 10))
corr_matrix = df.select_dtypes(include=[np.number]).corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

## 4. Train-Test Split

Given the temporal nature of the data (Years 2019-2021), we should ideally split by time to prevent data leakage.
We will use 2019 and 2020 for training, and 2021 for testing.

In [ ]:
# Define Features and Target
features = ['Year', 'Month', 'Province_Encoded', 'District_Encoded', 
            'Latitude', 'Longitude', 'Elevation', 
            'Temp_avg', 'Precipitation_avg', 'Humidity_avg']
target = 'Cases'

# Split based on Year
train_data = df[df['Year'] < 2021]
test_data = df[df['Year'] == 2021]

X_train = train_data[features]
y_train = train_data[target]

X_test = test_data[features]
y_test = test_data[target]

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

## 5. Model Training (XGBoost)

We use XGBoost Regressor to predict the number of cases.

In [ ]:
# Initialize XGBoost Regressor
model = xgb.XGBRegressor(objective='reg:squarederror', 
                         n_estimators=100, 
                         learning_rate=0.1, 
                         max_depth=5, 
                         random_state=42)

# Train the model
model.fit(X_train, y_train)

print("Model training completed.")

## 6. Evaluation

Evaluate the model using RMSE and MAE metrics.

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# --- Regression Metrics ---
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("--- Regression Metrics ---")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared Score: {r2:.2f}\n")

# --- Classification Metrics (High Case Outbreak Detection) ---
# Defining an outbreak threshold (e.g., 75th percentile of training data)
threshold = y_train.quantile(0.75)
print(f"Outbreak Threshold (75th percentile): {threshold:.2f} cases")

y_test_binary = (y_test > threshold).astype(int)
y_pred_binary = (y_pred > threshold).astype(int)

accuracy = accuracy_score(y_test_binary, y_pred_binary)
precision = precision_score(y_test_binary, y_pred_binary)
recall = recall_score(y_test_binary, y_pred_binary)
f1 = f1_score(y_test_binary, y_pred_binary)
auc = roc_auc_score(y_test_binary, y_pred)

print("--- Classification Metrics ---")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"AUC Score: {auc:.2f}")

# Visualize Results
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Actual vs Predicted Scatter Plot
sns.scatterplot(x=y_test, y=y_pred, alpha=0.5, ax=axes[0])
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Cases')
axes[0].set_ylabel('Predicted Cases')
axes[0].set_title('Actual vs Predicted Dengue Cases')

# 2. Confusion Matrix for Classification
cm = confusion_matrix(y_test_binary, y_pred_binary)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1])
axes[1].set_xlabel('Predicted Outbreak')
axes[1].set_ylabel('Actual Outbreak')
axes[1].set_title('Confusion Matrix (Outbreak vs No Outbreak)')

plt.tight_layout()
plt.show()

## 7. Model Interpretation (SHAP)

Using SHAP (SHapley Additive exPlanations) to understand feature importance and model behavior.

In [ ]:
# Initialize SHAP explainer
explainer = shap.Explainer(model)
shap_values = explainer(X_test)

# Summary Plot (Bee swarm)
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test)

# Feature Importance Bar Plot
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
# Dependence Plot for a key feature (e.g., Temp_avg)
# This shows how a specific feature interacts with the model's output
shap.dependence_plot("Temp_avg", shap_values.values, X_test)

In [ ]:
# Individual Prediction Explanation
print("--- Explaining Individual Predictions ---\n")

# 1. Select a few interesting samples (e.g., top 3 predicted cases and 2 random samples)
top_indices = np.argsort(y_pred)[-3:][::-1]
random_indices = np.random.choice(range(len(X_test)), 2, replace=False)
sample_indices = list(top_indices) + list(random_indices)

for idx in sample_indices:
    actual = y_test.iloc[idx]
    predicted = y_pred[idx]
    district = le_district.inverse_transform([X_test.iloc[idx]['District_Encoded']])[0]
    month = X_test.iloc[idx]['Month']
    
    print(f"--- Sample (Index {idx}) in {district} (Month: {month}) ---")
    print(f"Actual Cases: {actual}, Predicted Cases: {predicted:.2f}")
    
    # Get SHAP values for this instance
    instance_shap = shap_values[idx]
    
    # Display Waterfall Plot for visual explanation
    plt.figure(figsize=(10, 4))
    shap.waterfall_plot(instance_shap)
    plt.show()
    
    # Generate Text Explanation
    # Sort features by contribution
    contributions = []
    for i, feature in enumerate(features):
        val = instance_shap.values[i]
        contributions.append((feature, val))
    
    # Sort by absolute contribution strength
    contributions.sort(key=lambda x: abs(x[1]), reverse=True)
    
    explanation = f"Explaining Prediction: The model predicted {predicted:.2f} cases. "
    top_pos = [f"{f} (+{v:.2f})" for f, v in contributions if v > 0][:2]
    top_neg = [f"{f} ({v:.2f})" for f, v in contributions if v < 0][:2]
    
    if top_pos:
        explanation += f"The main factors INCREASING the prediction were: {', '.join(top_pos)}. "
    if top_neg:
        explanation += f"The main factors DECREASING the prediction were: {', '.join(top_neg)}."
    
    print(f"Reasoning: {explanation}\n")